**teneva** (examples)

This python package, named teneva (**ten**sor **eva**luation), provides very compact implementation for the multidimensional cross approximation algorithm in the tensor-train (TT) format. This package also contains a function for quickly calculating the values of the constructed low-rank tensor approximation, as well as a number of auxiliary useful utilities.

> See [github repo](https://github.com/AndreiChertkov/teneva) for more details and the License file.

---

In [9]:
import numpy as np
import teneva
from time import perf_counter as tpc
np.random.seed(42)

# Example for tt-cross (TODO)

In [7]:
Y = teneva.rand([4, 5, 7], 2)
teneva.show(Y)

  4  5  7 
 / \/ \/ \
 1  2  2  1 



# Example for maxvol

In [12]:
from maxvolpy.maxvol import maxvol as base_maxvol

In [13]:
N = 5000
r = 50
A = 2. * np.random.rand(N, r) - 1.
e = 1.01
K = 500

In [14]:
t = tpc()
I, B = teneva.maxvol(A, e, K)
t = tpc() - t

In [15]:
t0 = tpc()
I0, B0 = base_maxvol(A, tol=e, max_iters=K)
t0 = tpc() - t0

In [16]:
ei = np.max(np.abs(I - I0))
eb = np.max(np.abs(B - B0))

In [17]:
print(f'Time own   : {t:-8.4f}')
print(f'Time base  : {t0:-8.4f}')
print(f'Error I    : {ei:-8.2e}')
print(f'Error B    : {eb:-8.2e}')
print(f'Det own    : {np.linalg.det(A[I, :]):-8.2e}')
print(f'Det base   : {np.linalg.det(A[I0, :]):-8.2e}')
print(f'Max B own  : {np.max(np.abs(B)):-8.2e}')
print(f'Max B base : {np.max(np.abs(B0)):-8.2e}')

Time own   :   0.3136
Time base  :   0.0333
Error I    : 0.00e+00
Error B    : 3.11e-15
Det own    : -9.24e+26
Det base   : -9.24e+26
Max B own  : 1.00e+00
Max B base : 1.00e+00


In [19]:
tr, ei, eb = [], [], []
for i in range(10):
    A = 2. * np.random.rand(N, r) - 1.
    t = tpc(); I, B = teneva.maxvol(A, e, K); t = tpc() - t
    t0 = tpc(); I0, B0 = base_maxvol(A, tol=e, max_iters=K); t0 = tpc() - t0
    tr.append(t / t0)
    ei.append(np.max(np.abs(I - I0)))
    eb.append(np.max(np.abs(B - B0)))

print(f'Mean (time own / time base) : {np.mean(tr):-8.3f}')
print(f'Max error for I             : {np.max(ei):-8.2e}')
print(f'Max error for B             : {np.max(eb):-8.2e}')

Mean (time own / time base) :   15.824
Max error for I             : 0.00e+00
Max error for B             : 4.47e-15


# Example for rect_maxvol

In [24]:
from maxvolpy.maxvol import rect_maxvol as base_rect_maxvol

In [25]:
N = 5000
r = 50
A = 2. * np.random.rand(N, r) - 1.
e = 1.01
N_min = r + 10
N_max = r + 30

# Special function syntax from maxvolpy package:
maxK = N_max           
min_add_K = N_min - r
minK = None

In [26]:
t = tpc()
I, B = teneva.rect_maxvol(A, e, N_min, N_max)
t = tpc() - t

In [28]:
t0 = tpc()
I0, B0 = base_rect_maxvol(A, e, maxK, min_add_K, minK)
t0 = tpc() - t0

In [29]:
ei = np.max(np.abs(I - I0))
eb = np.max(np.abs(B - B0))

In [30]:
print(f'Time own   : {t:-8.4f}')
print(f'Time base  : {t0:-8.4f}')
print(f'Error I    : {ei:-8.2e}')
print(f'Error B    : {eb:-8.2e}')
print(f'Max B elem : {np.max(np.abs(B)):-8.2e}')

Time own   :   0.3760
Time base  :   0.0374
Error I    : 0.00e+00
Error B    : 2.53e-15
Max B elem : 1.00e+00


In [31]:
tr, ei, eb = [], [], []
for i in range(10):
    A = 2. * np.random.rand(N, r) - 1.
    t = tpc()
    I, B = teneva.rect_maxvol(A, e, N_min, N_max)
    t = tpc() - t
    t0 = tpc()
    I0, B0 = base_rect_maxvol(A, e, maxK, min_add_K, minK)
    t0 = tpc() - t0
    tr.append(t / t0)
    ei.append(np.max(np.abs(I - I0)))
    eb.append(np.max(np.abs(B - B0)))
    
print(f'Mean (time own / time base) : {np.mean(tr):-8.3f}')
print(f'Max error for I             : {np.max(ei):-8.2e}')
print(f'Max error for B             : {np.max(eb):-8.2e}')

Mean (time own / time base) :   15.794
Max error for I             : 0.00e+00
Max error for B             : 3.03e-15


# Example for cross 2D (draft)

In [20]:
M = 1000
N = 500
r = 100
idx = np.random.choice(r, size=M)
A = np.random.randn(r, N)[idx]
# A = np.random.randn(M, r) @ np.random.randn(r, N)

In [21]:
t = tpc()
U1, S1, V1 = np.linalg.svd(A)
t = tpc() - t

A1 = U1[:, :r] @ np.diag(S1[:r]) @ V1[:r, :]
e = np.linalg.norm(A - A1) / np.linalg.norm(A)

print(f'Error                   : {e:-8.2e}')
print(f'Time (sec)              : {t:-8.4f}')

Error                   : 2.47e-15
Time (sec)              :   0.1998


In [22]:
t = tpc()
I, J = None, np.arange(r)
for q in range(5):
    I, B = teneva.maxvol(A[:, J], e=1.01, K=500)
    J, B = teneva.maxvol(A[I, :].T, e=1.01, K=500)

C, U, R = A[:, J], np.linalg.inv(A[np.ix_(I, J)]), A[I, :]
t = tpc() - t

A2 = C @ U @ R
# e = np.linalg.norm(A - A2) / np.linalg.norm(A)
# e = np.linalg.norm((A - A2).ravel()  , np.inf)
e = np.max(np.abs(A - A2))

print(f'Error                   : {e:-8.2e}')
print(f'Time (sec)              : {t:-8.4f}')

Error                   : 1.75e-14
Time (sec)              :   0.1462


In [23]:
M = 1000
N = 2000
r = 20
e = []
for i in range(100):
    A = np.random.randn(M, r) @ np.random.rand(r, N)
    I = np.random.choice(M, size=r, replace=False)
    J = np.random.choice(N, size=r, replace=False)
    C = A[:, J]
    U = np.linalg.inv(A[np.ix_(I, J)])
    R = A[I, :]
    B = C @ U @ R
    e.append(np.max(np.abs(A-B)))
print(f'Error max : {np.max(e):-8.2e}')

Error max : 2.91e-10


---